# Ver2. raw data

## sparsity 고려하지 않고 origin rating data를 이용해 LightFM 모델을 학습해봅니다.
---
### 1. rating 결측치 처리 - 평균으로 대체
### 2. whisky columns 수정 (avr_rating, total_rating)
### 3. model 학습
- 기존에는 item_features는 학습했으나, 사용자의 선호 맛을 학습하지 못했습니다.
- 이를 보완하기 위해 user_features를 추가하여 모델을 학습하겠습니다.

In [2]:
import os
import pandas as pd
import numpy as  np
import random
from tqdm.notebook import tqdm

In [3]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\Wizzle\\S08P22A805\\model\\v2'

In [3]:
rating = pd.read_csv("../dataset/rating.csv", index_col = 0, encoding="utf-8")
whisky = pd.read_csv("../dataset/whisky.csv", index_col=0 , encoding="UTF-8")
whisky_index = pd.read_csv("../dataset/whisky_index.csv", index_col=0 , encoding="UTF-8")
user_index = pd.read_csv("../dataset/user_index.csv", index_col=0 , encoding="UTF-8")

## rating 결측치 처리 - 평균으로 대체합니다.
---
### 1. interactions 결측치 처리
### 2. whisky columns(avr_rating) 수정

## 1. interactions 결측치 처리

In [5]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908176 entries, 0 to 11258
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    908176 non-null  int64  
 1   whisky_id  908176 non-null  int64  
 2   rating     888132 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 27.7 MB


In [6]:
# Reset the index of the DataFrame
rating = rating.reset_index(drop=True)

# Group by "whisky_name" and fill missing values in "rating" with the rounded mean of that group
rating["rating"] = rating.groupby("whisky_id")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_7684\1581247862.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rating["rating"] = rating.groupby("whisky_id")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))


In [7]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908176 entries, 0 to 908175
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    908176 non-null  int64  
 1   whisky_id  908176 non-null  int64  
 2   rating     908176 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 20.8 MB


## 2. whisky 평균 평점, 총 인원 수정

In [8]:
whisky["avr_rating"] = round(rating.groupby("whisky_id")["rating"].mean(), 2)

In [9]:
whisky["total_rating"] = rating.groupby("whisky_id")['user_id'].count()

In [10]:
whisky

,whisky_id,link,image,name,avr_rating,category,location,total_rating,cost_rank,abv,...,herbal,oily,full_bodied,rich,sweet,salty,vanilla,tart,fruity,floral
0,0,/spirits/hibiki-21-year,https://ip-distiller.imgix.net/images/spirits/...,Hibiki 21 Year,9.01,Blended,Japan,861.0,5,43.00,...,30,20,80,80,85,15,20,25,85,50
1,1,/spirits/highland-park-18,https://ip-distiller.imgix.net/images/spirits/...,Highland Park 18 Year,8.89,Peated Single Malt,"Islands, Scotland",2988.0,4,43.00,...,20,40,70,80,70,40,50,50,70,20
2,2,/spirits/michter-s-20-year-kentucky-straight-b...,https://ip-distiller.imgix.net/images/spirits/...,Michter's 20 Year Kentucky Straight Bourbon (2...,9.00,Bourbon,"Kentucky, USA",10.0,5,57.10,...,0,15,80,90,85,5,30,25,35,0
3,3,/spirits/george-t-stagg-bourbon-fall-2019,https://ip-distiller.imgix.net/images/spirits/...,George T. Stagg Bourbon (Fall 2019),9.06,Bourbon,"Kentucky, USA",629.0,4,58.45,...,50,20,60,60,45,0,60,60,45,0
4,4,/spirits/bowmore-mizunara-cask-finish,https://ip-distiller.imgix.net/images/spirits/...,Bowmore Mizunara Cask Finish,7.86,Peated Single Malt,"Islay, Scotland",22.0,5,53.90,...,30,10,75,75,60,20,30,20,50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,3530,/spirits/wild-turkey-spiced,https://ip-distiller.imgix.net/images/spirits/...,Wild Turkey Spiced,5.92,Flavored Whiskey,"Kentucky, USA",13.0,1,43.00,...,0,0,40,55,85,0,25,10,10,0
3531,3531,/spirits/seagram-s-seven-crown-american-blende...,https://ip-distiller.imgix.net/images/spirits/...,Seagram's 7 Crown American Blended Whiskey,4.31,Blended American Whiskey,USA,237.0,1,40.00,...,20,0,30,0,100,0,100,0,60,10
3532,3532,/spirits/11-wells-single-malt-whiskey,https://ip-distiller.imgix.net/images/spirits/...,11 Wells Single Malt Whiskey,6.00,American Single Malt,"Minnesota, USA",2.0,3,42.00,...,10,40,10,10,90,0,50,20,30,10
3533,3533,/spirits/immortal-spirits-early-whiskey,https://ip-distiller.imgix.net/images/spirits/...,Immortal Spirits Early Whiskey,2.00,Other Whiskey,"Oregon, USA",1.0,2,44.50,...,100,80,0,0,40,0,20,0,0,0


In [11]:
whisky.iloc[0]

whisky_id                                                       0
link                                      /spirits/hibiki-21-year
image           https://ip-distiller.imgix.net/images/spirits/...
name                                               Hibiki 21 Year
avr_rating                                                   9.01
category                                                  Blended
location                                                    Japan
total_rating                                                861.0
cost_rank                                                       5
abv                                                          43.0
cask_type       ex-bourbon American oak, ex-sherry European oa...
smoky                                                          30
peaty                                                          85
spicy                                                          50
herbal                                                         30
oily      

### 결측치 평균으로 대체 완료

In [12]:
rating.to_csv("dataset/rating.csv", encoding="UTF-8")
whisky.to_csv("dataset/whisky.csv", encoding="UTF-8")

## 2. 정규화 진행하지 않고 학습 진행

In [13]:
import os

from scipy.sparse import coo_matrix, csr_matrix
from tqdm.notebook import tqdm

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k, reciprocal_rank


C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### 2-1. item meta data 생성

In [14]:
columns = whisky.columns.tolist()
columns = [columns[8]] + columns[11:]
columns

['cost_rank',
 'smoky',
 'peaty',
 'spicy',
 'herbal',
 'oily',
 'full_bodied',
 'rich',
 'sweet',
 'salty',
 'vanilla',
 'tart',
 'fruity',
 'floral']

In [15]:
item_features = np.zeros((whisky['whisky_id'].nunique(), len(columns)))
for i in range(whisky['whisky_id'].nunique()):
    for j, col_name in enumerate(columns):
        item_features[i, j] = whisky.iloc[i][col_name]
# item_features = csr_matrix(item_features)

In [16]:
item_features

array([[ 5., 30., 85., ..., 25., 85., 50.],
       [ 4., 40., 30., ..., 50., 70., 20.],
       [ 5., 15.,  0., ..., 25., 35.,  0.],
       ...,
       [ 3., 90., 10., ..., 20., 30., 10.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1., 31.,  0., ...,  0., 30., 10.]])

In [17]:
item_features = csr_matrix(item_features)

In [18]:
item_features.shape

(3535, 14)

In [19]:
item_feature_cols = ["cost_rank", "smoky", "peaty", "spicy", "herbal", "oily", "full_bodied", "rich", "sweet", "salty", "vanilla", "tart", "fruity", "floral"]
item_df = pd.DataFrame(data=item_features.toarray(), columns=item_feature_cols)

In [20]:
item_df.to_csv("dataset/item_features.csv", encoding="UTF-8")

### 2-2. user_features 만들기

In [21]:
## user 별로 max rating score 측정
max_rating = rating.groupby("user_id").max(lambda x: max(x))["rating"]

In [22]:
max_rating

user_id
0          9.0
1          9.0
2          9.0
3         10.0
4         10.0
          ... 
119510    10.0
119511     8.0
119512     6.0
119513     2.0
119514     8.0
Name: rating, Length: 119515, dtype: float64

In [23]:
user_feature_ids = []
for i in range(rating["user_id"].nunique()):
    user_feature_ids.append([])
    
    
for i, row in tqdm(rating.iterrows()):
    if int(row["rating"]) == max_rating[int(row["user_id"])]: user_feature_ids[int(row["user_id"])].append(int(row["whisky_id"]))

0it [00:00, ?it/s]

In [24]:
def make_user_features(user_feature_ids, data):
    user_features = []
    for i, info in enumerate(user_feature_ids):
        whisky_id = info[random.randint(0, len(info)-1)]
        print(f"{i} user - {whisky_id}")
        whisky_info = whisky.iloc[whisky_id]
        user_features.append([whisky_info[8]] + list(whisky_info[11:]))
    return user_features

In [25]:
user_features = make_user_features(user_feature_ids, whisky)

0 user - 811
1 user - 1081
2 user - 0
3 user - 796
4 user - 0
5 user - 0
6 user - 0
7 user - 70
8 user - 0
9 user - 0
10 user - 556
11 user - 1676
12 user - 1107
13 user - 593
14 user - 0
15 user - 0
16 user - 0
17 user - 1368
18 user - 37
19 user - 0
20 user - 0
21 user - 0
22 user - 25
23 user - 0
24 user - 0
25 user - 0
26 user - 0
27 user - 0
28 user - 0
29 user - 94
30 user - 0
31 user - 2657
32 user - 0
33 user - 1368
34 user - 0
35 user - 0
36 user - 584
37 user - 0
38 user - 470
39 user - 0
40 user - 557
41 user - 0
42 user - 0
43 user - 0
44 user - 0
45 user - 0
46 user - 0
47 user - 1342
48 user - 1107
49 user - 37
50 user - 1922
51 user - 94
52 user - 0
53 user - 0
54 user - 1
55 user - 0
56 user - 861
57 user - 1341
58 user - 43
59 user - 0
60 user - 1832
61 user - 1616
62 user - 2823
63 user - 0
64 user - 0
65 user - 0
66 user - 37
67 user - 2832
68 user - 0
69 user - 2669
70 user - 170
71 user - 0
72 user - 0
73 user - 37
74 user - 0
75 user - 0
76 user - 392
77 user - 16

1836 user - 1097
1837 user - 609
1838 user - 95
1839 user - 811
1840 user - 1663
1841 user - 1
1842 user - 991
1843 user - 1349
1844 user - 1465
1845 user - 87
1846 user - 2233
1847 user - 1
1848 user - 1
1849 user - 92
1850 user - 804
1851 user - 2473
1852 user - 360
1853 user - 73
1854 user - 1
1855 user - 33
1856 user - 1066
1857 user - 1974
1858 user - 521
1859 user - 1
1860 user - 1
1861 user - 715
1862 user - 2670
1863 user - 938
1864 user - 609
1865 user - 1
1866 user - 1
1867 user - 1
1868 user - 1340
1869 user - 1
1870 user - 817
1871 user - 1
1872 user - 249
1873 user - 1
1874 user - 1672
1875 user - 2670
1876 user - 1081
1877 user - 1
1878 user - 39
1879 user - 12
1880 user - 1
1881 user - 1106
1882 user - 609
1883 user - 1
1884 user - 2201
1885 user - 1989
1886 user - 1560
1887 user - 251
1888 user - 1
1889 user - 2235
1890 user - 1
1891 user - 815
1892 user - 1
1893 user - 341
1894 user - 2128
1895 user - 1
1896 user - 96
1897 user - 1
1898 user - 37
1899 user - 2247
1900 

3859 user - 322
3860 user - 455
3861 user - 1342
3862 user - 2524
3863 user - 3
3864 user - 3
3865 user - 3
3866 user - 1661
3867 user - 1368
3868 user - 2379
3869 user - 593
3870 user - 3
3871 user - 596
3872 user - 405
3873 user - 3
3874 user - 3
3875 user - 3
3876 user - 3
3877 user - 3
3878 user - 2254
3879 user - 3
3880 user - 3
3881 user - 786
3882 user - 557
3883 user - 182
3884 user - 3049
3885 user - 409
3886 user - 891
3887 user - 3
3888 user - 735
3889 user - 811
3890 user - 3
3891 user - 462
3892 user - 3
3893 user - 56
3894 user - 817
3895 user - 401
3896 user - 1951
3897 user - 3
3898 user - 401
3899 user - 3
3900 user - 3
3901 user - 397
3902 user - 602
3903 user - 613
3904 user - 3
3905 user - 42
3906 user - 3
3907 user - 3
3908 user - 593
3909 user - 1368
3910 user - 3
3911 user - 3
3912 user - 3
3913 user - 1348
3914 user - 816
3915 user - 3
3916 user - 3
3917 user - 1035
3918 user - 3088
3919 user - 3
3920 user - 3301
3921 user - 314
3922 user - 602
3923 user - 3
392

5662 user - 12
5663 user - 12
5664 user - 37
5665 user - 1075
5666 user - 589
5667 user - 12
5668 user - 12
5669 user - 2654
5670 user - 12
5671 user - 92
5672 user - 1665
5673 user - 504
5674 user - 2860
5675 user - 1669
5676 user - 1368
5677 user - 2183
5678 user - 13
5679 user - 602
5680 user - 247
5681 user - 13
5682 user - 584
5683 user - 574
5684 user - 1984
5685 user - 13
5686 user - 13
5687 user - 1104
5688 user - 13
5689 user - 13
5690 user - 13
5691 user - 16
5692 user - 409
5693 user - 593
5694 user - 609
5695 user - 13
5696 user - 2454
5697 user - 401
5698 user - 598
5699 user - 13
5700 user - 584
5701 user - 394
5702 user - 1368
5703 user - 593
5704 user - 1092
5705 user - 13
5706 user - 37
5707 user - 1616
5708 user - 1368
5709 user - 13
5710 user - 1979
5711 user - 203
5712 user - 13
5713 user - 955
5714 user - 13
5715 user - 614
5716 user - 13
5717 user - 15
5718 user - 1802
5719 user - 1342
5720 user - 401
5721 user - 2468
5722 user - 2211
5723 user - 13
5724 user - 13

7257 user - 36
7258 user - 90
7259 user - 1686
7260 user - 118
7261 user - 348
7262 user - 37
7263 user - 88
7264 user - 326
7265 user - 37
7266 user - 392
7267 user - 37
7268 user - 557
7269 user - 37
7270 user - 2453
7271 user - 37
7272 user - 94
7273 user - 37
7274 user - 37
7275 user - 452
7276 user - 1341
7277 user - 602
7278 user - 261
7279 user - 37
7280 user - 37
7281 user - 857
7282 user - 238
7283 user - 445
7284 user - 811
7285 user - 37
7286 user - 1047
7287 user - 678
7288 user - 1084
7289 user - 37
7290 user - 37
7291 user - 1341
7292 user - 37
7293 user - 800
7294 user - 1347
7295 user - 1081
7296 user - 678
7297 user - 695
7298 user - 37
7299 user - 219
7300 user - 57
7301 user - 37
7302 user - 1679
7303 user - 37
7304 user - 37
7305 user - 401
7306 user - 1238
7307 user - 1104
7308 user - 37
7309 user - 978
7310 user - 1922
7311 user - 615
7312 user - 37
7313 user - 37
7314 user - 811
7315 user - 1107
7316 user - 37
7317 user - 37
7318 user - 37
7319 user - 591
7320 us

...

112616 user - 2753
112617 user - 2753
112618 user - 2753
112619 user - 2753
112620 user - 3520
112621 user - 2753
112622 user - 2753
112623 user - 2753
112624 user - 2845
112625 user - 2753
112626 user - 2860
112627 user - 2753
112628 user - 2753
112629 user - 2753
112630 user - 2753
112631 user - 2753
112632 user - 2753
112633 user - 2753
112634 user - 2753
112635 user - 2753
112636 user - 2754
112637 user - 2754
112638 user - 2754
112639 user - 2754
112640 user - 2754
112641 user - 2754
112642 user - 2754
112643 user - 2754
112644 user - 2754
112645 user - 2754
112646 user - 2754
112647 user - 2755
112648 user - 2755
112649 user - 2756
112650 user - 2757
112651 user - 3369
112652 user - 2759
112653 user - 2759
112654 user - 2759
112655 user - 2759
112656 user - 2759
112657 user - 2759
112658 user - 2759
112659 user - 2759
112660 user - 2759
112661 user - 2760
112662 user - 2760
112663 user - 2760
112664 user - 3077
112665 user - 2760
112666 user - 2762
112667 user - 2762
112668 user 

114030 user - 2858
114031 user - 2858
114032 user - 2858
114033 user - 2859
114034 user - 2859
114035 user - 3278
114036 user - 2859
114037 user - 2859
114038 user - 2859
114039 user - 2859
114040 user - 2859
114041 user - 2860
114042 user - 2860
114043 user - 2860
114044 user - 2860
114045 user - 2860
114046 user - 2860
114047 user - 2860
114048 user - 2860
114049 user - 2860
114050 user - 2860
114051 user - 2860
114052 user - 2860
114053 user - 2860
114054 user - 2860
114055 user - 2860
114056 user - 2860
114057 user - 2860
114058 user - 2860
114059 user - 2860
114060 user - 2860
114061 user - 2860
114062 user - 2860
114063 user - 2860
114064 user - 2860
114065 user - 2860
114066 user - 2860
114067 user - 2860
114068 user - 2860
114069 user - 2860
114070 user - 2860
114071 user - 2860
114072 user - 2860
114073 user - 2860
114074 user - 2860
114075 user - 2860
114076 user - 2860
114077 user - 2860
114078 user - 2860
114079 user - 2860
114080 user - 2860
114081 user - 2860
114082 user 

115540 user - 2996
115541 user - 2996
115542 user - 2996
115543 user - 2996
115544 user - 2996
115545 user - 2996
115546 user - 2996
115547 user - 2999
115548 user - 2999
115549 user - 3000
115550 user - 3001
115551 user - 3000
115552 user - 3000
115553 user - 3000
115554 user - 3000
115555 user - 3000
115556 user - 3000
115557 user - 3000
115558 user - 3000
115559 user - 3000
115560 user - 3001
115561 user - 3001
115562 user - 3077
115563 user - 3001
115564 user - 3001
115565 user - 3001
115566 user - 3001
115567 user - 3001
115568 user - 3001
115569 user - 3001
115570 user - 3001
115571 user - 3001
115572 user - 3002
115573 user - 3002
115574 user - 3003
115575 user - 3003
115576 user - 3004
115577 user - 3004
115578 user - 3004
115579 user - 3004
115580 user - 3004
115581 user - 3005
115582 user - 3005
115583 user - 3005
115584 user - 3005
115585 user - 3005
115586 user - 3012
115587 user - 3005
115588 user - 3005
115589 user - 3005
115590 user - 3005
115591 user - 3005
115592 user 

117369 user - 3223
117370 user - 3223
117371 user - 3223
117372 user - 3223
117373 user - 3223
117374 user - 3419
117375 user - 3223
117376 user - 3223
117377 user - 3223
117378 user - 3223
117379 user - 3223
117380 user - 3223
117381 user - 3223
117382 user - 3223
117383 user - 3223
117384 user - 3223
117385 user - 3223
117386 user - 3223
117387 user - 3223
117388 user - 3223
117389 user - 3223
117390 user - 3223
117391 user - 3223
117392 user - 3223
117393 user - 3223
117394 user - 3223
117395 user - 3223
117396 user - 3223
117397 user - 3223
117398 user - 3223
117399 user - 3223
117400 user - 3223
117401 user - 3223
117402 user - 3223
117403 user - 3223
117404 user - 3223
117405 user - 3223
117406 user - 3223
117407 user - 3223
117408 user - 3223
117409 user - 3223
117410 user - 3223
117411 user - 3223
117412 user - 3223
117413 user - 3223
117414 user - 3223
117415 user - 3223
117416 user - 3223
117417 user - 3223
117418 user - 3223
117419 user - 3223
117420 user - 3223
117421 user 

119056 user - 3419
119057 user - 3419
119058 user - 3419
119059 user - 3419
119060 user - 3419
119061 user - 3419
119062 user - 3419
119063 user - 3419
119064 user - 3419
119065 user - 3419
119066 user - 3419
119067 user - 3419
119068 user - 3419
119069 user - 3419
119070 user - 3420
119071 user - 3420
119072 user - 3421
119073 user - 3421
119074 user - 3421
119075 user - 3421
119076 user - 3421
119077 user - 3421
119078 user - 3421
119079 user - 3421
119080 user - 3421
119081 user - 3421
119082 user - 3421
119083 user - 3421
119084 user - 3421
119085 user - 3421
119086 user - 3421
119087 user - 3421
119088 user - 3421
119089 user - 3421
119090 user - 3421
119091 user - 3421
119092 user - 3421
119093 user - 3421
119094 user - 3421
119095 user - 3422
119096 user - 3422
119097 user - 3422
119098 user - 3422
119099 user - 3422
119100 user - 3422
119101 user - 3422
119102 user - 3422
119103 user - 3422
119104 user - 3422
119105 user - 3422
119106 user - 3422
119107 user - 3422
119108 user 

In [26]:
user_features = csr_matrix(user_features)
user_features.shape

(119515, 14)

### user features 완성

In [27]:
user_feature_cols = ["cost_rank", "smoky", "peaty", "spicy", "herbal", "oily", "full_bodied", "rich", "sweet", "salty", "vanilla", "tart", "fruity", "floral"]
user_df = pd.DataFrame(data=user_features.toarray(), columns=user_feature_cols)

In [28]:
user_df.to_csv("dataset/user_features.csv", encoding="UTF-8")

### user_item_interactions 만들기

In [29]:
user_index['user_id'].nunique()

119515

In [30]:
whisky_index['whisky_id'].nunique()

3535

In [31]:
interactions = coo_matrix(
    (
        rating['rating'].values,
        (rating['user_id'].values, rating["whisky_id"].values)
    ),
    shape=(user_index['user_id'].nunique(), whisky_index['whisky_id'].nunique())
)

In [32]:
print(interactions)
interactions

  (0, 0)	9.0
  (1, 0)	7.0
  (2, 0)	9.0
  (3, 0)	10.0
  (4, 0)	9.0
  (5, 0)	7.0
  (6, 0)	10.0
  (7, 0)	7.0
  (8, 0)	10.0
  (9, 0)	10.0
  (10, 0)	9.0
  (11, 0)	9.0
  (12, 0)	8.0
  (13, 0)	8.0
  (14, 0)	10.0
  (15, 0)	10.0
  (16, 0)	9.0
  (17, 0)	9.0
  (18, 0)	8.0
  (19, 0)	9.0
  (20, 0)	9.0
  (21, 0)	10.0
  (22, 0)	9.0
  (23, 0)	9.0
  (24, 0)	9.0
  :	:
  (7636, 3531)	4.0
  (56292, 3531)	2.0
  (2670, 3531)	2.0
  (786, 3531)	2.0
  (5942, 3531)	6.0
  (15561, 3531)	2.0
  (25623, 3532)	6.0
  (2885, 3532)	6.0
  (7368, 3533)	2.0
  (69554, 3534)	10.0
  (4484, 3534)	2.0
  (85727, 3534)	5.0
  (43900, 3534)	4.0
  (16070, 3534)	8.0
  (29998, 3534)	8.0
  (7895, 3534)	4.0
  (104052, 3534)	5.0
  (31152, 3534)	2.0
  (119512, 3534)	6.0
  (2509, 3534)	4.0
  (4828, 3534)	4.0
  (119513, 3534)	2.0
  (119514, 3534)	8.0
  (3123, 3534)	4.0
  (95648, 3534)	2.0


<119515x3535 sparse matrix of type '<class 'numpy.float64'>'
	with 908176 stored elements in COOrdinate format>

In [33]:
interactions.toarray()

array([[9., 0., 0., ..., 0., 0., 0.],
       [7., 0., 0., ..., 0., 0., 0.],
       [9., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 6.],
       [0., 0., 0., ..., 0., 0., 2.],
       [0., 0., 0., ..., 0., 0., 8.]])

In [34]:
density = interactions.nnz / (interactions.shape[0] * interactions.shape[1])
sparsity_ratio = 1 - (interactions.nnz / (interactions.shape[0] * interactions.shape[1]))

print("Density:", density)
print("Sparsity ratio:", sparsity_ratio)

Density: 0.0021496026402324673
Sparsity ratio: 0.9978503973597675
